<a href="https://colab.research.google.com/github/Antonio24ch/Materia_apps_ciencia_de_datos/blob/main/ingesta_almacenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install pickle
!pip install yaml
!pip install sodapy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

In [ ]:
with open('credentials.yaml', 'r') as f:
    x= yaml.safe_load(f)


FileNotFoundError: ignored



1.   get_client: Esta función regresa un cliente que se puede conectar a la API de inspecciones de establecimiento dándole un token previamente generado.
 


In [ ]:
def get_client():
    client = Socrata("data.cityofchicago.org", 
    token = x['api_chicago']['app_token'],
    username = x['api_chicago']['username'],
    password = x['api_chicago']['password'])

       
    return client





2.   ingesta_inicial: Esta función recibe como parámetros el cliente con el que nos podemos comunicar con la API, y el límite de registros que queremos obtener al llamar a la API. Regresa una lista de los elementos que la API regresó.
 



In [ ]:
def ingesta_inicial(cliente,limite):
  data = "4ijn-s7e5"
  limite = limite
  cliente = client.get(data, limit=limite)
  result = cliente
  return result

In [ ]:
ingesta = ingesta_inicial(client, 300000)

In [ ]:
ingesta[33]

{'inspection_id': '2571968',
 'dba_name': 'VILLAGE HOUSE,LLC',
 'aka_name': 'VILLAGE HOUSE',
 'license_': '2452040',
 'facility_type': 'Daycare (2 - 6 Years)',
 'risk': 'Risk 1 (High)',
 'address': '9510 S CONSTANCE AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60617',
 'inspection_date': '2023-03-02T00:00:00.000',
 'inspection_type': 'License',
 'results': 'Fail',
 'violations': "1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - Comments: PIC DOESN'T HAVE A VALID CITY CERTIFIED FOOD MANAGERS CERTIFICATE.MUST PROVIDE AND MAINTAIN.(PRIORITY FOUNDATION 7-38-012) | 2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - Comments: OBSERVED NO VALID CITY FOOD MANAGERS CERTIFICATE POSTED TO VIEW ONLY A COPY WHILE TCS FOODS ARE BEING HANDLED AND SERVED SUCH AS (CHICKEN). MUST PROVIDE ORIGINAL AND MAINTAIN.(PRIORITY FOUNDATION  7-38-012)(CITATION ISSUED) | 3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments

In [ ]:
results_df = pd.DataFrame.from_records(ingesta)
results_df.head(5)

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,results,violations,latitude,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu
0,2572122,PRETTYBOY CAFE LLC,ESTEREO,2863407,Restaurant,Risk 2 (Medium),1001 W FULTON MARKET,CHICAGO,IL,60607,...,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.88662880065414,-87.6522287277881,"{'latitude': '41.88662880065414', 'longitude':...",41,14917,29,63,46
1,2572120,STAND-UP BURGER,STAND-UP BURGER,2560625,Restaurant,Risk 1 (High),204 N WELLS ST,CHICAGO,IL,60606,...,Out of Business,NaN,41.88586331626135,-87.63408111682442,"{'latitude': '41.88586331626135', 'longitude':...",22,14914,38,92,36
2,2572085,PENELOPE'S VEGAN TAQUERIA #2,PENELOPE'S VEGAN TAQUERIA #2,2895971,Restaurant,Risk 1 (High),5204 N CLARK ST,CHICAGO,IL,60640,...,Pass,51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICE...,41.97635748112271,-87.66856777865902,"{'latitude': '41.97635748112271', 'longitude':...",46,22616,76,564,24
3,2572124,PRETTYBOY CAFE LLC,ESTEREO,2863408,Restaurant,Risk 2 (Medium),1001 W FULTON MARKET,CHICAGO,IL,60607,...,Fail,NaN,41.88662880065414,-87.6522287277881,"{'latitude': '41.88662880065414', 'longitude':...",41,14917,29,63,46
4,2572094,THORNTONS #322,THORNTON'S,2535182,Grocery Store,Risk 2 (Medium),3450 S CALIFORNIA AVE,CHICAGO,IL,60608,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.830402822601805,-87.69495246786789,"{'latitude': '41.830402822601805', 'longitude'...",29,22248,55,639,1




3.   get_s3_resource: Esta función regresa un resource de S3 para poder guardar datos en el bucket.


In [ ]:
def get_s3_resource(aws_access_key_id, aws_secret_access_key):
    s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    return s3

In [ ]:
aws_access_key_id = x['s3']['aws_access_key_id']
aws_secret_access_key = x['s3']['aws_secret_access_key']
get_s3_resource(aws_access_key_id, aws_secret_access_key)

s3.ServiceResource()



4.   
guardar_ingesta: Esta función recibe como parámetros el nombre del bucket de S3, la ruta  en el bucket en donde se guardarán los datos y el data frame que se guardará como un pickle en esa ruta.
 


In [ ]:
def guardar_ingesta(bucket_name, bucket_path, df):
    session = boto3.Session(
        aws_access_key_id = x['s3']['aws_access_key_id'],
        aws_secret_access_key = x['s3']['aws_secret_access_key'],
        aws_session_token= x['s3']['aws_session_token']
    )

    s3 = session.resource('s3')
    nombre_bucket = "aplicaciones-cd-1" + x['iexe']['matricula']
    data = pickle.dumps(ingesta)
    bucket_name = nombre_bucket
    ruta =  "ingesta/inicial/inspecciones-historicas-" + str(TODAY) + ".pkl"
    s3.Object(nombre_bucket, bucket_path).put(Body=df)

In [ ]:
session = boto3.Session(
    aws_access_key_id = x['s3']['aws_access_key_id'],
    aws_secret_access_key = x['s3']['aws_secret_access_key'],
    aws_session_token= x['s3']['aws_session_token']
  )
s3 = session.resource('s3')
nombre_bucket = "aplicaciones-cd-1" + x['iexe']['matricula']
s3.create_bucket(Bucket=nombre_bucket)

s3.Bucket(name='aplicaciones-cd-1mcda22b051')

In [ ]:
data = pickle.dumps(ingesta)
bucket_name = nombre_bucket
ruta =  "ingesta/inicial/inspecciones-historicas-" + str(TODAY) + ".pkl"
guardar_ingesta(bucket_name, ruta, data)
print("La ingesta ha sido guardada exitosamente en el bucket")


La ingesta ha sido guardada exitosamente en el bucket




1.   ingesta_consecutiva: Esta función recibe como parámetros el cliente con el que nos podemos comunicar con la API, la fecha de la que se quieren obtener nuevos datos al llamar a la API y el límite de registros para obtener de regreso.



In [ ]:
def ingesta_consecutiva(cliente, fecha, limite):
    data = "4ijn-s7e5"
    limite = limite
    fecha= "inspection_date>='{}'".format(fecha)
    resultados = cliente.get(data,where=fecha, limit=limite)
    return resultados

In [ ]:
result_= ingesta_consecutiva(client, '2020-11-03', 1000)

In [ ]:
result_[33]

{'inspection_id': '2545883',
 'dba_name': 'COLOR CLUB',
 'aka_name': 'COLOR CLUB',
 'license_': '2809101',
 'facility_type': 'Restaurant',
 'risk': 'Risk 3 (Low)',
 'address': '4108-4116 N HAMLIN AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60618',
 'inspection_date': '2021-12-21T00:00:00.000',
 'inspection_type': 'License',
 'results': 'Fail',
 'latitude': '41.95584984486689',
 'longitude': '-87.72291941423855',
 'location': {'latitude': '41.95584984486689',
  'longitude': '-87.72291941423855'}}

In [ ]:
results_2 = pd.DataFrame.from_records(result_)
results_2.head(5)

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2568436,SCHWAB REHAB HOSPITAL&CARE NET,SCHWAB REHAB HOSPITAL&CARE NET,2205586,Hospital,Risk 1 (High),1401 S CALIFORNIA AVE,CHICAGO,IL,60608,2022-12-13T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
1,2567065,PARK MANOR,PARK MANOR ELEMENTARY,24841,School,Risk 1 (High),7037 S Rhodes (532E) AVE,CHICAGO,IL,60637,2022-11-15T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
2,2564901,CHEEZE AND THANK YOU,CHEEZE AND THANK YOU,2872888,NaN,All,2046 E GRAND AVE,NaN,IL,NaN,2022-10-06T00:00:00.000,License,Not Ready,NaN,NaN,NaN,NaN
3,2561538,"PACINO'S RC, LLC",PACINOS,2683667,Restaurant,Risk 1 (High),1010 S DELANO CT,CHICAGO,IL,60605,2022-08-01T00:00:00.000,Canvass,No Entry,NaN,NaN,NaN,NaN
4,2555843,MC DONALD'S 6039,MC DONALD'S,2341076,Restaurant,Risk 2 (Medium),5656 W IRVING PARK RD,CHICAGO,IL,60634,2022-05-16T00:00:00.000,Canvass,Pass,41.95326311074034,-87.76931122260137,"{'latitude': '41.95326311074034', 'longitude':...",NaN


In [ ]:
def guardar_consecutiva(bucket_name, bucket_path, df):
    session = boto3.Session(
        aws_access_key_id = x['s3']['aws_access_key_id'],
        aws_secret_access_key = x['s3']['aws_secret_access_key'],
        aws_session_token= x['s3']['aws_session_token']
    )

    s3 = session.resource('s3')
    nombre_bucket = "aplicaciones-cd-1" + x['iexe']['matricula']
    data = pickle.dumps(result_)
    bucket_name = nombre_bucket
    ruta =  "ingesta/consecutiva/inspecciones-historicas-" + str(TODAY) + ".pkl"
    s3.Object(nombre_bucket, bucket_path).put(Body=df)

In [ ]:
data = pickle.dumps(result_)
bucket_name = nombre_bucket
ruta =  "ingesta/consecutiva/inspecciones-historicas-" + str(TODAY) + ".pkl"
guardar_consecutiva(bucket_name, ruta, data)
print("La ingesta ha sido guardada exitosamente en el bucket")

La ingesta ha sido guardada exitosamente en el bucket
